In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV

In [ ]:
# Charger les données
data = pd.read_csv(r"C:\Users\yzi\OneDrive\Bureau\Travaux\Projet_Classification_THBS\database\credit_card_fraud_corr.csv", index_col="transactionId")

# Afficher les premières lignes du DataFrame
print(data.head())

In [ ]:
# Remplacer les lettres "C" et "M" par des chiffres dans les colonnes "nameOrig" et "nameDest"
def replace_first_letter(value):
    if value.startswith('C'):
        return '1' + value[1:]
    elif value.startswith('M'):
        return '2' + value[1:]
    else:
        return value


#data['nameOrig'] = data['nameOrig'].apply(replace_first_letter)
#data['nameDest'] = data['nameDest'].apply(replace_first_letter)
#data['nameOrig'] = data['nameOrig'].astype('float')
#data['nameDest'] = data['nameDest'].astype('float')

# Convertir une colonne en type 'category'
data['isFraud'] = data['isFraud'].astype('category')

# Créer un dictionnaire de correspondance entre les types et les valeurs numériques
type_mapping = {'TRANSFER': 1, 'PAYMENT': 2, 'CASH_OUT': 3, 'CASH_IN': 4, 'DEBIT': 5}

# Remplacer les valeurs de la colonne "type" par les valeurs numériques correspondantes
data['type'] = data['type'].replace(type_mapping)


In [ ]:
# Séparer les caractéristiques et la variable cible
X = data.drop("isFraud", axis=1)  # Utiliser "isFraud" au lieu de "Class"
y = data["isFraud"]

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer l'objet SMOTE
smote = SMOTE(random_state=42, k_neighbors=4)

# Appliquer SMOTE sur les données d'entraînement uniquement
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Définir les hyperparamètres à rechercher
param_grid = {
    'n_estimators': [ 1000, 1400],
    'max_depth': [ 7, 9]
}

# Instancier le modèle XGBClassifier
model = XGBClassifier(enable_categorical=True)

# Créer l'objet GridSearchCV avec SMOTE
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=5)

# Effectuer la recherche sur la grille avec les données équilibrées
grid_search.fit(X_train_resampled, y_train_resampled)

# Afficher les meilleurs hyperparamètres
print("Meilleurs hyperparamètres trouvés:")
print(grid_search.best_params_)

# Obtenir les meilleures performances
best_model = grid_search.best_estimator_

# Faire des prédictions sur les données de test
y_pred = best_model.predict(X_test)

# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

# Définir le nombre de plis pour la validation croisée
n_splits = 10

# Instancier la validation croisée stratifiée
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Liste pour stocker les rapports de classification de chaque pli
cv_classification_reports = []

# Boucle sur les plis de la validation croisée
for train_idx, val_idx in stratified_kfold.split(X, y):
    # Diviser les données en ensembles d'entraînement et de validation
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
    
    # Appliquer SMOTE sur les données d'entraînement uniquement
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train_fold, y_train_fold)
    
    # Entraîner le modèle sur l'ensemble d'entraînement
    model.fit(X_train_resampled, y_train_resampled)
    
    # Faire des prédictions sur l'ensemble de validation
    y_pred_val = model.predict(X_val_fold)
    
    # Générer un rapport de classification pour ce pli
    classification_report_fold = classification_report(y_val_fold, y_pred_val, output_dict=True)
    
    # Ajouter le rapport à la liste des rapports
    cv_classification_reports.append(classification_report_fold)

# Fusionner les rapports de tous les plis
overall_classification_report = {
    "0": {"precision": 0, "recall": 0, "f1-score": 0, "support": 0},
    "1": {"precision": 0, "recall": 0, "f1-score": 0, "support": 0}
}
for report in cv_classification_reports:
    for label in overall_classification_report.keys():
        overall_classification_report[label]["precision"] += report[label]["precision"]
        overall_classification_report[label]["recall"] += report[label]["recall"]
        overall_classification_report[label]["f1-score"] += report[label]["f1-score"]
        overall_classification_report[label]["support"] += report[label]["support"]

# Calculer les moyennes des rapports de classification
num_folds = len(cv_classification_reports)
for label in overall_classification_report.keys():
    overall_classification_report[label]["precision"] /= num_folds
    overall_classification_report[label]["recall"] /= num_folds
    overall_classification_report[label]["f1-score"] /= num_folds

# Afficher le rapport de classification global
print("Overall classification report:")
print(overall_classification_report)


In [ ]:
import pickle

# Définir le nom du fichier pour sauvegarder le modèle
filename = 'XGBmodel.pkl'

# Utiliser pickle.dump() pour sauvegarder le modèle dans le fichier
with open(filename, 'wb') as file:
    pickle.dump(model, file)

print("Le modèle a été sauvegardé avec succès dans le fichier:", filename)


In [ ]:
from sklearn.metrics import confusion_matrix

# Faire des prédictions sur les données de test
y_pred = best_model.predict(X_test)

# Calculer la matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred)

# Extraire les éléments de la matrice de confusion
tn, fp, fn, tp = conf_matrix.ravel()

# Calculer les statistiques du modèle
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)

# Afficher la matrice de confusion et les statistiques du modèle
print("Matrice de confusion:")
print(conf_matrix)
print("Faux positifs:", fp)
print("Faux négatifs:", fn)
print("Accuracy:", accuracy)
print("Précision:", precision)
print("Rappel:", recall)
print("F1-score:", f1_score)
